### Consulta com expressão regular

In [1]:
from google.cloud import bigquery
import os  
from os.path import join
import re
import xml.etree.cElementTree as ET
import requests;

base_url = "https://api.stackexchange.com";
exception = "Exception";

repostioryName = "../../Dataset_crashanalysis/Github_issues";
files_folders = [];
current_folder = "";
expression = re.compile('([A-Za-z.]+(Error|Exception|Iteration)([^\n]*))');
add = ET.Element("add");

### Obtém diretorios

In [2]:
folders = [name for name in os.listdir(repostioryName)];
folders= sorted(folders, key=str.lower, reverse=True);
def get_next_folder():
    return folders.pop();

### Obtém arquivos

In [3]:
def get_next_file():
    global files_folders;
    global current_folder;
    
    if len(files_folders) <= 0:
        current_folder = get_next_folder();
        files_folders = [name for name in os.listdir(join(repostioryName, current_folder))];
        files_folders = sorted(files_folders, key=str.lower, reverse=True);
        
    return join(repostioryName, current_folder, files_folders.pop());

### Adiciona elemento ao xml

In [4]:
def add_element(request, response):
    doc = ET.SubElement(add, "doc");
    ET.SubElement(doc, "field", name="Exception_Parse").text = request;
    ET.SubElement(doc, "field", name="Post_link").text = response['link'];

### Busca das respostas

In [5]:
def search(exeptions):
    for exeption in exeptions:
        url = f"https://api.stackexchange.com/2.3/search/advanced?order=desc&sort=activity&body={exeption}&site=stackoverflow"
        r = requests.get(url);
        data = r.json();
        print(data)
        if not 'error_id' in data:
            data = [k for k in data['items'] if ('java' or 'android' in k['tags']) and (k['is_answered'] == True)];
            for item in data:
                add_element(exeption, item);

### Trata com expressão regular

In [6]:
fileName = get_next_file();
while fileName:
    f = open(fileName, "r")
    text = f.read();
    result = expression.findall(text);
    exeptions = [re.sub('\'.*?\' ','',item[0], flags=re.DOTALL) for item in result];
    search(exeptions);
    fileName = get_next_file();

{'items': [{'tags': ['java', 'android', 'sqlite', 'android-studio'], 'owner': {'account_id': 3833650, 'reputation': 21, 'user_id': 3178565, 'user_type': 'registered', 'accept_rate': 25, 'profile_image': 'https://www.gravatar.com/avatar/b55a94c582e1f7ff002c07c05a0ba6cc?s=128&d=identicon&r=PG&f=1', 'display_name': 'ClemDC', 'link': 'https://stackoverflow.com/users/3178565/clemdc'}, 'is_answered': False, 'view_count': 28, 'answer_count': 0, 'score': 0, 'last_activity_date': 1564785802, 'creation_date': 1564785802, 'question_id': 57334163, 'content_license': 'CC BY-SA 4.0', 'link': 'https://stackoverflow.com/questions/57334163/updating-sqlite-db-and-getting-nullpointerexception', 'title': 'updating sqlite db and getting nullpointerexception'}, {'tags': ['java', 'android', 'firebase'], 'owner': {'account_id': 15134919, 'reputation': 53, 'user_id': 10921697, 'user_type': 'registered', 'profile_image': 'https://www.gravatar.com/avatar/1f846b29f28d6f95e281a38e9a3c7dd4?s=128&d=identicon&r=PG&f=

{'error_id': 502, 'error_message': 'too many requests from this IP, more requests available in 80230 seconds', 'error_name': 'throttle_violation'}
{'error_id': 502, 'error_message': 'too many requests from this IP, more requests available in 80230 seconds', 'error_name': 'throttle_violation'}
{'error_id': 502, 'error_message': 'too many requests from this IP, more requests available in 80229 seconds', 'error_name': 'throttle_violation'}
{'error_id': 502, 'error_message': 'too many requests from this IP, more requests available in 80229 seconds', 'error_name': 'throttle_violation'}
{'error_id': 502, 'error_message': 'too many requests from this IP, more requests available in 80228 seconds', 'error_name': 'throttle_violation'}
{'error_id': 502, 'error_message': 'too many requests from this IP, more requests available in 80227 seconds', 'error_name': 'throttle_violation'}
{'error_id': 502, 'error_message': 'too many requests from this IP, more requests available in 80227 seconds', 'error_

KeyboardInterrupt: 

### Salva XML

In [ ]:
tree = ET.ElementTree(add);
tree.write("../output.xml");